In [10]:
from bs4 import BeautifulSoup
import requests

uri  = "https://iopscience.iop.org/nsearch?terms=ZnTe&nextPage=2&previousPage=-1&currentPage=1&searchDatePeriod=anytime&orderBy=relevance&pageLength=50"

r1 = requests.get(uri)
coverpage = r1.content

In [11]:
soup1 = BeautifulSoup(coverpage, 'lxml')

In [99]:
td = soup1.find_all('div', {"class" : "art-list-item-body"})

In [100]:
len(td)

50

In [111]:
articulos = []
otras     = []
for i in td:
    a  = i.findChild('div', {"class" : "article-text view-text-small"}).findChild("p")
    b  = i.findChild('h2',  {"class" : "art-list-item-title"}).findChild("a")
    c  = i.find_all('p', {"class" : "small art-list-item-meta"})
    articulos.append([a.text ,b.text])
    otras.append(c)

In [125]:
otras[5][1].find_all('span', {"itemprop" : "author"})

[<span itemprop="author" itemtype="http://schema.org/Person">O I Olusola</span>,
 <span itemprop="author" itemtype="http://schema.org/Person">H I Salim</span>,
 <span itemprop="author" itemtype="http://schema.org/Person">I M Dharmadasa</span>]

In [128]:
otras[5][1].find_all('em')

[<em> Mater. Res. Express</em>]

In [131]:
otras[20][1].text

'\r\n            2009 Nanotechnology 20 305702\r\n            \u2003\r\n            https://doi.org/10.1088/0957-4484/20/30/305702\n'